In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 28049721
paper_name = 'yofe_thoms_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [57]:
original_data = pd.read_excel('raw_data/TablesS1-S7.xlsx', sheet_name='Table S1', skiprows=1)

In [58]:
original_data.head()

,ORF,Gene,Deletion / DAmP,Number of single cells analyzed,Mean number of peroxisomes identified per cell [a.u],Z score of mean number of peroxisomes identified per cell,SEM number of peroxisomes identified per cell,Mean size of identified peroxisomes [a.u],Z score mean size of identified peroxisomes,SEM size of identified peroxisomes
0,YDL065C,PEX19,Deletion,74,0.61,-7.58,0.09,17.92,3.24,6.89
1,YKR001C,VPS1,Deletion,72,0.99,-6.05,0.06,23.83,5.75,2.18
2,YOL044W,PEX15,Deletion,89,1.11,-5.54,0.08,9.18,-0.46,1.21
3,YNL329C,PEX6,Deletion,148,1.12,-5.50,0.07,11.47,0.51,1.16
4,YOR193W,PEX27,Deletion,62,1.16,-5.34,0.12,18.18,3.36,1.46


In [59]:
original_data['Deletion / DAmP'].unique()

array(['Deletion', 'DAmP'], dtype=object)

In [60]:
original_data = original_data.loc[original_data['Deletion / DAmP']=='Deletion',:].copy()

In [61]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4219 x 10


In [62]:
original_data['orf'] = original_data['ORF'].astype(str)

In [63]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [64]:
typo_fixes = {'YOLO57W':'YOL057W','YOLO62C':'YOL062C',
              'YKLO72W':'YKL072W','YJL206-A':'YJL206C-A','YLR287-A':'YLR287C-A'}

In [65]:
for s in typo_fixes.keys():
    original_data.loc[original_data['orf']==s,'orf'] = typo_fixes[s]

In [71]:
v[np.isnan(v)]

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [72]:
# Translate to ORFs 
v = translate_sc(original_data['orf'].values, to='orf')
original_data['orf2'] = v

In [73]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf2'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene, Deletion / DAmP, Number of single cells analyzed, Mean number of peroxisomes identified per cell [a.u], Z score of mean number of peroxisomes identified per cell, SEM number of peroxisomes identified per cell, Mean size of identified peroxisomes [a.u], Z score mean size of identified peroxisomes, SEM size of identified peroxisomes, orf, orf2]
Index: []


In [74]:
original_data.set_index('orf2', inplace=True)
original_data.index.name='orf'

In [75]:
original_data = original_data[['Mean number of peroxisomes identified per cell [a.u]',
                              'Mean size of identified peroxisomes [a.u]']]

In [76]:
original_data = original_data.groupby(original_data.index).mean()

In [77]:
original_data.shape

(4195, 2)

# Prepare the final dataset

In [78]:
data = original_data.copy()

In [79]:
dataset_ids = [11818, 11819]
datasets = datasets.reindex(index=dataset_ids)

In [80]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [81]:
data.head()

dataset_id,11818,11819
data_type,value,value
orf,,
YAL002W,2.24,6.66
YAL004W,2.40,8.65
YAL005C,2.17,8.77
YAL007C,2.33,7.55
YAL008W,2.56,7.08


## Subset to the genes currently in SGD

In [82]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [83]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [84]:
data.head()

,dataset_id,11818,11819
,data_type,value,value
gene_id,orf,,
2,YAL002W,2.24,6.66
1863,YAL004W,2.40,8.65
4,YAL005C,2.17,8.77
5,YAL007C,2.33,7.55
6,YAL008W,2.56,7.08


# Normalize

In [85]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [86]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [87]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [88]:
data_all.head()


,dataset_id,11818,11819,11818,11819
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,2.24,6.66,-0.776024,-0.728633
1863,YAL004W,2.40,8.65,-0.116216,-0.005615
4,YAL005C,2.17,8.77,-1.064690,0.037984
5,YAL007C,2.33,7.55,-0.404882,-0.405273
6,YAL008W,2.56,7.08,0.543592,-0.576036


# Print out

In [89]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [90]:
from IO.save_data_to_db3 import *

In [91]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 28049721...
Inserting the new data...


100%|██████████| 2/2 [00:12<00:00,  6.31s/it]

Updating the data_modified_on field...
